# Lab 20B

In the first lab, it will recognise plate numbers. Run and see the magic! This code will work on VS Code in either MS Windows or Raspberry Pi (Linux).

# Installers - Run only once BEFORE you start your program (if needed)

In [ ]:
#The following are pre-requisites for the code. Run it only once in the Pi 400

import platform
import os

# Detect the operating system
current_platform = platform.system()
print(current_platform)
if current_platform == "Windows":
    # Windows-specific commands
    
    os.system('pip install djitellopy opencv-python pytesseract pillow openai==0.28 opencv-python-headless langchain langchain_openai google-cloud-vision')
    
    print("Installation completed for Windows. Note: tesseract-ocr may require manual installation on Windows.")
elif current_platform == "Linux":  # Pi 400
    # Pi-specific commands

    os.system('sudo pip3 install opencv-python djitellopy pytesseract opencv-python opencv-python-headless openai openai==0.28 langchain langchain_openai google-cloud-vision --break-system-packages')
    os.system('sudo apt-get install tesseract-ocr -y')         
    os.system('sudo apt-get install libgtk2.0-dev pkg-config && sudo apt-get install libgtk-3-dev -y')  
    
    os.system('wget -P /home/pi/Desktop/Lab20 https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml')
    print("Installation completed for Raspberry Pi.")
else:
    print("Unsupported platform. Please ensure you are running this on Windows or Raspberry Pi.")



# Section 1 - Number Plate Recognition System

1. The following code is the code for a number plate recognition system. If the tello drone is connected, it will activate the drone's camera. If not, then it will activate the usb camera attached to your computer. You can also upload a picture of a numberplate to be analysed. It uses the Russian numberplate XML which might result in inaccuracies. Run this code, then comment on the accuracy of the recognition.

In [ ]:
#Revision 3

from tkinter import *
from tkinter import filedialog
import cv2
from PIL import Image, ImageTk
import pytesseract
from djitellopy import Tello
import platform
import tempfile
import requests

# Set the path to Tesseract OCR and will work for both windows and Linux
if platform.system() == "Windows":
    # Set the path for Windows
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
else:
    # Set the path for Linux (Raspberry Pi typically uses the system-installed tesseract)
    pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'



def is_tello_connected():
    """
    Check if the Tello drone is connected.
    """
    try:
        tello = Tello()
        tello.connect()
        tello.streamon()
        return True, tello
    except Exception as e:
        print(f"Tello connection failed: {e}")
        return False, None


def run_tello_gui(tello):
    """
    Runs the GUI for the Tello drone control and recognition.
    """
    frame_read = tello.get_frame_read()

    # Load the Haar cascade for number plate detection from the internet (instead of downloading the file)
    # URL to the Haar cascade file
    url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml"
    response = requests.get(url)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".xml") as temp_file:
        temp_file.write(response.content)
        temp_file_path = temp_file.name

# Load the cascade
    plate_cascade = cv2.CascadeClassifier(temp_file_path)
    #plate_cascade = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')

    def takeoff():
        tello.takeoff()
        takeoff_button.config(state=DISABLED)
        land_button.config(state=NORMAL)
        up_button.config(state=NORMAL)
        down_button.config(state=NORMAL)
        left_button.config(state=NORMAL)
        right_button.config(state=NORMAL)
        capture_button.config(state=NORMAL)

    def land():
        tello.land()
        takeoff_button.config(state=NORMAL)
        land_button.config(state=DISABLED)
        up_button.config(state=DISABLED)
        down_button.config(state=DISABLED)
        left_button.config(state=DISABLED)
        right_button.config(state=DISABLED)
        capture_button.config(state=NORMAL)

    def up():
        tello.move_up(100)

    def down():
        tello.move_down(100)

    def left():
        tello.move_left(100)

    def right():
        tello.move_right(100)

    def capture_and_display():
        # Capture frame from Tello stream
        frame = frame_read.frame

        if frame is not None and frame.size != 0:
            # Convert to grayscale for detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect number plates in the image
            plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(30, 30))

            recognized_text = ""

            for (x, y, w, h) in plates:
                # Draw a rectangle around each detected number plate on the frame
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # Extract the plate region for OCR
                plate_region = frame[y:y + h, x:x + w]

                # Use Tesseract to recognize text in the plate region
                plate_text = pytesseract.image_to_string(plate_region, config='--psm 8')
                recognized_text += plate_text.strip() + "\n"  # Add each plate's text to output

            # Display the detected text in the GUI
            text_label.config(text=recognized_text)

            # Convert the frame to RGB and resize for Tkinter
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(frame_rgb)
            img_pil = img_pil.resize((200, 150))

            # Convert to PhotoImage for Tkinter
            img_tk = ImageTk.PhotoImage(img_pil)

            # Update the image on the GUI
            display_label.config(image=img_tk)
            display_label.image = img_tk  # Keep a reference to avoid garbage collection
        else:
            print("Failed to capture frame from the Tello stream.")

    # GUI setup for Tello
    window = Tk()
    window.geometry("500x400")
    window.title("Tello Drone - Number Plate Recognition")

    # Buttons
    global takeoff_button, land_button, up_button, down_button, left_button, right_button, capture_button

    takeoff_button = Button(window, text="Take Off", command=takeoff, state=NORMAL)
    takeoff_button.place(x=20, y=20, width=120, height=40)

    land_button = Button(window, text="Land", command=land, state=DISABLED)
    land_button.place(x=160, y=20, width=120, height=40)

    up_button = Button(window, text="Up", command=up, state=DISABLED)
    up_button.place(x=90, y=80, width=120, height=40)

    down_button = Button(window, text="Down", command=down, state=DISABLED)
    down_button.place(x=90, y=140, width=120, height=40)

    left_button = Button(window, text="Left", command=left, state=DISABLED)
    left_button.place(x=20, y=200, width=120, height=40)

    right_button = Button(window, text="Right", command=right, state=DISABLED)
    right_button.place(x=160, y=200, width=120, height=40)

    capture_button = Button(window, text="Capture", command=capture_and_display, state=NORMAL)
    capture_button.place(x=90, y=260, width=120, height=40)

    # Image display label
    display_label = Label(window)
    display_label.place(x=320, y=20, width=150, height=150)

    # Label to display recognized number plate text
    text_label = Label(window, text="", font=("Helvetica", 10), fg="blue", wraplength=150)
    text_label.place(x=320, y=200)

    window.mainloop()


def run_camera_gui():
    """
    Runs the fallback GUI for image-based recognition.
    """
    def upload_and_extract():
        try:
            # Open file dialog to select image
            file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
            if not file_path:
                result_label.config(text="No file selected.")
                return

            # Open the image using PIL
            image = Image.open(file_path)

            # Use Tesseract to extract text
            extracted_text = pytesseract.image_to_string(image, config="--psm 8")  # PSM 8: Assume single word/block
            result_label.config(text=f"Extracted Number Plate: {extracted_text.strip()}")
        except Exception as e:
            result_label.config(text=f"Error: {e}")

    # GUI setup for image upload
    window = Tk()
    window.title("Fallback - Number Plate Recognition")
    window.geometry("400x200")

    instruction_label = Label(window, text="Click 'Upload Image' to select a picture of a car.")
    instruction_label.pack(pady=10)

    upload_button = Button(window, text="Upload Image", command=upload_and_extract)
    upload_button.pack(pady=10)

    result_label = Label(window, text="", fg="blue", wraplength=350)
    result_label.pack(pady=10)

    

    window.mainloop()


# Main Program Logic
tello_connected, tello = is_tello_connected()
if tello_connected:
    run_tello_gui(tello)
else:
    print("Tello drone not detected. Running fallback mode.")
    run_camera_gui()


2. Describe what happens when you run the code above (try with USB camera attached, and without camera attached. When camera not present, connect to drone.). Comment also on the accuracy of the image detection. Is it good? Is it bad? How do you improve the detection accuracy? Write your answer in the space below:

# Section 2 - Kernel killer - Only Use when Drone program is not responding.

This code will kill the kernel - causing it to reset. Use when drone program is not responding.

In [5]:
#This program will kill the Kernel causing it to reset. Use when necessary (such as between code executions - when the drone is not responding)
def killall(): 
    import os
    import platform
    import subprocess
    import sys

    def reset_kernel():
    
        #Detect the operating system and reset the Jupyter Notebook kernel accordingly.
    
        os_name = platform.system()
        print(f"Detected OS: {os_name}")

        if os_name == "Linux":  # Assuming Raspberry Pi 400 runs a Linux-based OS
            try:
            # Run the kill command for Python processes in Linux
                print("Resetting kernel for Pi 400 (Linux)...")
                result = subprocess.run("kill -9 $(ps -A | grep python | awk '{print $1}')", shell=True, check=True)
                print("Kernel reset successfully!")
            except subprocess.CalledProcessError as e:
                print(f"Failed to reset kernel on Pi 400: {e}")
        elif os_name == "Windows":
            try:
            # Use os._exit to exit the Python process on Windows
                print("Resetting kernel for Windows...")
                os._exit(0)
            except Exception as e:
                print(f"Failed to reset kernel on Windows: {e}")
        else:
            print("Unsupported operating system.")

# Run the kernel reset function
    reset_kernel()


# Section 3 - Object detection using AI (OpenAI) and Tello Drone

3. The last code is on object detection using AI. This code can detect what the Drone sees and describe it. Run the program below with multiple objects, then comment on the accuracy of the AI in the space below:

In [ ]:
from tkinter import *
import cv2
from PIL import Image, ImageTk
import base64
import requests
import os
from djitellopy import Tello



api_key = os.getenv("OPENAI_API_KEY")


# Initialize Tello and connect
tello = Tello()
tello.connect()
tello.streamon()
frame_read = tello.get_frame_read()

def takeoff():
    print("Taking off")
    tello.takeoff()

def land():
    print("Landing")
    tello.land()

def up():
    print("Moving up")
    tello.move_up(100)

def down():
    print("Moving down")
    tello.move_down(100)

def left():
    print("Moving left")
    tello.move_left(100)

def right():
    print("Moving right")
    tello.move_right(100)

def capture_and_save_image():
    # Capture frame from Tello
    frame = frame_read.frame
    if frame is not None:
        cv2.imwrite("picture.jpg", frame)
        print("Image saved as picture.jpg")
        recognize_image_with_openai("picture.jpg")

        # Convert the frame to RGB for display
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(frame_rgb)

        # Resize the image for display in the GUI
        img_pil = img_pil.resize((400, 300))  # Resize as needed
        img_tk = ImageTk.PhotoImage(img_pil)

        display_label.config(image=img_tk)
        display_label.image = img_tk  # Keep a reference to avoid garbage collection

    else:
        print("Failed to capture image")

def recognize_image_with_openai(image_path):
    # Convert image to Base64
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')
    
    # Set up API request
    headers = {
        "Content-Type": "application/json",
        
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o-mini",  # Adjust model as required
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Describe the content of the image"},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 300
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    if response.status_code == 200:
        description = response.json()['choices'][0]['message']['content']
        print("Description:", description)
        text_label = Label(window, text="", font=("Helvetica", 10), fg="blue", wraplength=400, justify="left")
        text_label.place(x=50, y=170)
        text_label.config(text=description)
    else:
        print("Failed to get a response:", response.json())

# GUI setup
window = Tk()
window.geometry("500x350")

button1 = Button(window, text="Take Off", command=takeoff)
button1.place(x=120, y=20, width=120, height=25)

button2 = Button(window, text="Land", command=land)
button2.place(x=120, y=80, width=120, height=25)

button3 = Button(window, text="Up", command=up)
button3.place(x=120, y=0, width=120, height=25)

button4 = Button(window, text="Down", command=down)
button4.place(x=120, y=100, width=120, height=25)

button5 = Button(window, text="Left", command=left)
button5.place(x=75, y=50, width=120, height=25)

button6 = Button(window, text="Right", command=right)
button6.place(x=150, y=50, width=120, height=25)

capture_button = Button(window, text="Capture Photo", command=capture_and_save_image)
capture_button.place(x=120, y=150, width=120, height=25)

text_label = Label(window, text="", font=("Helvetica", 10), fg="blue")
text_label.place(x=250, y=170)

display_label = Label(window)
display_label.pack(pady=10)

window.mainloop()
